In [50]:
#conda install langchain -c conda-forge
#conda install langchain langchain-community langchain-chroma -c conda-forge

from langchain_community.llms import Ollama
llm = Ollama(model="llama3")
response = llm.invoke("The first man on the moon was ...")
print(response)

Neil Armstrong! He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. His famous words upon setting foot on the moon were: "That's one small step for man, one giant leap for mankind."


Read transaction data

In [51]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/thu-vu92/local-llms-analyse-finance/refs/heads/main/transactions_2022_2023.csv")
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08
7,2023-12-22,Consulting,Income,541.57
8,2023-12-22,Aidsfonds,Expense,10.70
9,2023-12-20,Consulting,Income,2641.93


In [52]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [53]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

# Categorise bank transactions with Llama3

In [54]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [55]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Birtat Restaurant Amsterdam, Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [56]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['Here are the categorized expenses:', '', '* ISS Catering Services De Meern - Food', '* Taxi Utrecht - Transportation', '* Etos AMSTERDAM NLD - Shopping', '* Bistro Bar Amsterdam - Entertainment', '* Spotify AB by Adyen - Entertainment', '* Birtat Restaurant Amsterdam - Food', '* Boulders Ams Amsterdam Nld - Sport', '', 'Let me know if you have any other expenses that need categorizing!']


,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,* ISS Catering Services De Meern - Food,* ISS Catering Services De Meern,Food
3,* Taxi Utrecht - Transportation,* Taxi Utrecht,Transportation
4,* Etos AMSTERDAM NLD - Shopping,* Etos AMSTERDAM NLD,Shopping
5,* Bistro Bar Amsterdam - Entertainment,* Bistro Bar Amsterdam,Entertainment
6,* Spotify AB by Adyen - Entertainment,* Spotify AB by Adyen,Entertainment
7,* Birtat Restaurant Amsterdam - Food,* Birtat Restaurant Amsterdam,Food
8,* Boulders Ams Amsterdam Nld - Sport,* Boulders Ams Amsterdam Nld,Sport
9,,,None


In [57]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

categories_df_all

['Here are the categorized expenses:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Grocery', '3. Monthly Appartment Rent - Rent', '4. Vishandel Sier Amsterdam - Food', '5. Selling Paintings - Income', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Clothing', '8. Consulting - Work', '9. Aidsfonds - Charity', '10. Tls Bv Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Health', '12. Beta Boulders Ams Amsterdam - Sport', '13. Salary - Income', '14. Bouldermuur Bv Amsterdam - Sport', '15. Birtat Restaurant Amsterdam - Dining', '16. Freelancing - Work', '17. Tikkie - Expenses', '18. Blogging - Income', '19. Taxi Utrecht - Transportation', '20. Apple Services - Tech', '21. Amazon Lux - Shopping', '22. Classpass* Monthly - Fitness', '23. Audible Uk AdblCo/Pymt Gbr - Entertainment']


,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
3,2. Tesco Breda - Grocery,2. Tesco Breda,Grocery
4,3. Monthly Appartment Rent - Rent,3. Monthly Appartment Rent,Rent
5,4. Vishandel Sier Amsterdam - Food,4. Vishandel Sier Amsterdam,Food
6,5. Selling Paintings - Income,5. Selling Paintings,Income
7,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
8,7. Tk Maxx Amsterdam Da - Clothing,7. Tk Maxx Amsterdam Da,Clothing
9,8. Consulting - Work,8. Consulting,Work


In [58]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [59]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Taxes', 'Grocery', 'Rent', 'Food', 'Income',
       'Entertainment', 'Clothing', 'Work', 'Charity', 'Transportation',
       'Health', 'Sport', 'Dining', 'Expenses', 'Tech', 'Shopping',
       'Fitness'], dtype=object)

In [60]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Fitness", then categorise as "Sport and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Fitness"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [61]:
# Remove the numbering eg "1. " from Transaction column
# Index Reset

# Fixing the SettingWithCopyWarning
categories_df_all.loc[:, 'Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '', regex=True)

# Resetting the index
categories_df_all = categories_df_all.reset_index(drop=True)

# Display the DataFrame
categories_df_all

,Transaction vs category,Transaction,Category
0,1. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2. Tesco Breda - Grocery,Tesco Breda,Grocery
2,3. Monthly Appartment Rent - Rent,Monthly Appartment Rent,Rent
3,4. Vishandel Sier Amsterdam - Food,Vishandel Sier Amsterdam,Food and Drinks
4,5. Selling Paintings - Income,Selling Paintings,Income
5,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
6,7. Tk Maxx Amsterdam Da - Clothing,Tk Maxx Amsterdam Da,Clothing
7,8. Consulting - Work,Consulting,Work
8,9. Aidsfonds - Charity,Aidsfonds,Charity
9,10. Tls Bv Inz Ov-Chipkaart - Transportation,Tls Bv Inz Ov-Chipkaart,Transportation


In [64]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)

df = pd.read_csv("https://raw.githubusercontent.com/thu-vu92/local-llms-analyse-finance/refs/heads/main/transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,1. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,2. Tesco Breda - Grocery,Tesco Breda,Grocery
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,3. Monthly Appartment Rent - Rent,Monthly Appartment Rent,Rent
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,4. Vishandel Sier Amsterdam - Food,Vishandel Sier Amsterdam,Food and Drinks
4,2023-12-29,Selling Paintings,Income,13.63,5. Selling Paintings - Income,Selling Paintings,Income
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,7. Tk Maxx Amsterdam Da - Clothing,Tk Maxx Amsterdam Da,Clothing
7,2023-12-22,Consulting,Income,541.57,8. Consulting - Work,Consulting,Work
8,2023-12-22,Aidsfonds,Expense,10.70,9. Aidsfonds - Charity,Aidsfonds,Charity
9,2023-12-20,Consulting,Income,2641.93,8. Consulting - Work,Consulting,Work


In [67]:
df.to_csv('extract.csv')